In [9]:
import numpy as np

# ENCODING

In [64]:
def getTextToTransmit():
    with open("input.txt", "r") as f:
        text_to_transmit = f.read()
    return text_to_transmit;

def encodeText( text ):
    #each character is a vector of 8 bits
    return np.unpackbits(bytearray(text,"utf-8"))

def convoluteEncoding(bits):
    b=[1,1]+list(map(lambda x:(x-0.5)*2,bits))+[1,1]
    x=[xi for j in range(2,len(b)) for xi in [b[j]*b[j-2],b[j]*b[j-1]*b[j-2]]]
    return x

def scaleArrayKTimes(k,bits):
    return [b for b in bits for i in range(k)]

def addCalibrationZeros(k,bits):
    return np.concatenate((np.zeros(k),bits))

def writeChannelInput(bits):
    with open("encoded.txt", "w") as f:
        stringToSend = "\n".join(map(lambda bit:str(int(bit)),bits))
        #stringToSend = stringToSend[:-1]
        f.write(stringToSend)

In [65]:
MAX_SIZE=51200
NB_BYTES=80
BITS_PER_BYTES=8
ENCODER_FACTOR=2
NB_FINAL_DUMMY_BITS=2
BITS_ARRAY_SIZE=(NB_BYTES*BITS_PER_BYTES+NB_FINAL_DUMMY_BITS)*ENCODER_FACTOR
print("BITS_ARRAY_SIZE :",BITS_ARRAY_SIZE)

#for a mysterious reason the pipe cannot handle 51200 bits therefore we substract one on the repetition to lower our value
REPETITION_OFFSET=1

REPEAT_FACTOR= MAX_SIZE//(BITS_ARRAY_SIZE)-1-REPETITION_OFFSET
print("REPEAT_FACTOR :",REPEAT_FACTOR)

CALIBRATION_SIZE=BITS_ARRAY_SIZE
print('BITS_ARRAY_SIZE * REPEAT_FACTOR + CALIBRATION_SIZE =',BITS_ARRAY_SIZE,'*',REPEAT_FACTOR,'+',CALIBRATION_SIZE,'=',BITS_ARRAY_SIZE*REPEAT_FACTOR+CALIBRATION_SIZE)

x=addCalibrationZeros(CALIBRATION_SIZE,scaleArrayKTimes(REPEAT_FACTOR,convoluteEncoding(encodeText(getTextToTransmit()))))


print("input length :",len(x),"maximum is",MAX_SIZE)

BITS_ARRAY_SIZE : 1284
REPEAT_FACTOR : 37
BITS_ARRAY_SIZE * REPEAT_FACTOR + CALIBRATION_SIZE = 1284 * 37 + 1284 = 48792
input length : 48792 maximum is 51200


In [67]:
writeChannelInput(x)

# DECODING

In [68]:
def getChannelOutput():
    with open("received.txt", "r") as f:
        bitArray = np.array(f.read().split('\n'))
        return list(map(lambda x: float(x),bitArray[:-1]))
    
def getBitsFromY(Y):
    bias=np.mean(Y[:CALIBRATION_SIZE])
    print("bias =",bias)
    
    #group by bloc of size 38 starting at 1280 -> get 1280 bits
    bits=[np.mean(Y[CALIBRATION_SIZE+j*REPEAT_FACTOR:CALIBRATION_SIZE+(j+1)*REPEAT_FACTOR])-biais for j in range(BITS_ARRAY_SIZE)]
    
    return bits

def dot(a,b):
    return a[0]*b[0]+a[1]*b[1]

In [69]:
y=getBitsFromY(getChannelOutput())

bias = -0.027698537768870806
